**Set Environment**

In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [2]:
FD_INPS=($(ls -d ${FD_RES}/data/Input?*/))
for FD_INP in ${FD_INPS[@]}; do
    echo ${FD_INP}
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/


In [4]:
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?*,TFX?_DMSO,TFX?_Dex}/))
for FD_BED in ${FD_BEDS[@]}; do
    echo ${FD_BED}
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/TFX2_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/TFX2_DMSO/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/TFX3_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj

In [ ]:
should try using the fragment within count 

In [10]:
### set environment
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --mem 8G \
    -o ${FD_LOG}/fragment_input_score_per1_f50.txt \
    <<'EOF'
#!/bin/bash

### set directories & global variables
source ../config_duke.sh

#CHROMS=($(seq 1 22) X Y)
#CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=chr${CHROM}
CHROM=chr17
TARGET=target_PER1

FRAC=0.50
SUFFIX=f50

### Set input and output
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_INPS=($(ls -d ${FD_RES}/data/Input?*/))

FN_BED=${TARGET}.bed.gz
FN_INP=${CHROM}.bed


### START Message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

for FD_BED in ${FD_BEDS[@]}; do
    
    ### init
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/input_score/${SAMPLE}
    mkdir -p ${FD_OUT}
    
    ### Loop START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input file: " ${FD_BED}/${FN_BED}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3
    echo
    
    for FD_INP in ${FD_INPS[@]}; do
        
        ### get the input file and set output file
        INPUT=$(basename ${FD_INP})
        FN_OUT=${TARGET}_${SUFFIX}_${INPUT}.bed.gz
        echo "Output file: " ${FD_OUT}/${FN_OUT}
        
        ### intersect w/ input and count
        bedtools intersect \
            -a ${FD_BED}/${FN_BED} \
            -b ${FD_INP}/${FN_INP} \
            -f ${FRAC} -r -c |\
        gzip -c > ${FD_OUT}/${FN_OUT}
        
        ### Loop END Message
        echo "Show the first few lines of the output file"
        echo ${FD_OUT}/${FN_OUT}
        zcat ${FD_OUT}/${FN_OUT} | head -n 3
        echo
    done
done

### END message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27283777


In [11]:
### set environment
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --mem 8G \
    -o ${FD_LOG}/fragment_input_score_per1_f70.txt \
    <<'EOF'
#!/bin/bash

### set directories & global variables
source ../config_duke.sh

#CHROMS=($(seq 1 22) X Y)
#CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=chr${CHROM}
CHROM=chr17
TARGET=target_PER1

FRAC=0.70
SUFFIX=f70

### Set input and output
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_INPS=($(ls -d ${FD_RES}/data/Input?*/))

FN_BED=${TARGET}.bed.gz
FN_INP=${CHROM}.bed


### START Message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

for FD_BED in ${FD_BEDS[@]}; do
    
    ### init
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/input_score/${SAMPLE}
    mkdir -p ${FD_OUT}
    
    ### Loop START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input file: " ${FD_BED}/${FN_BED}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3
    echo
    
    for FD_INP in ${FD_INPS[@]}; do
        
        ### get the input file and set output file
        INPUT=$(basename ${FD_INP})
        FN_OUT=${TARGET}_${SUFFIX}_${INPUT}.bed.gz
        echo "Output file: " ${FD_OUT}/${FN_OUT}
        
        ### intersect w/ input and count
        bedtools intersect \
            -a ${FD_BED}/${FN_BED} \
            -b ${FD_INP}/${FN_INP} \
            -f ${FRAC} -r -c |\
        gzip -c > ${FD_OUT}/${FN_OUT}
        
        ### Loop END Message
        echo "Show the first few lines of the output file"
        echo ${FD_OUT}/${FN_OUT}
        zcat ${FD_OUT}/${FN_OUT} | head -n 3
        echo
    done
done

### END message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27283778


In [12]:
### set environment
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --mem 8G \
    -o ${FD_LOG}/fragment_input_score_per1_f90.txt \
    <<'EOF'
#!/bin/bash

### set directories & global variables
source ../config_duke.sh

#CHROMS=($(seq 1 22) X Y)
#CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=chr${CHROM}
CHROM=chr17
TARGET=target_PER1

FRAC=0.90
SUFFIX=f90

### Set input and output
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_INPS=($(ls -d ${FD_RES}/data/Input?*/))

FN_BED=${TARGET}.bed.gz
FN_INP=${CHROM}.bed


### START Message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

for FD_BED in ${FD_BEDS[@]}; do
    
    ### init
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/input_score/${SAMPLE}
    mkdir -p ${FD_OUT}
    
    ### Loop START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input file: " ${FD_BED}/${FN_BED}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3
    echo
    
    for FD_INP in ${FD_INPS[@]}; do
        
        ### get the input file and set output file
        INPUT=$(basename ${FD_INP})
        FN_OUT=${TARGET}_${SUFFIX}_${INPUT}.bed.gz
        echo "Output file: " ${FD_OUT}/${FN_OUT}
        
        ### intersect w/ input and count
        bedtools intersect \
            -a ${FD_BED}/${FN_BED} \
            -b ${FD_INP}/${FN_INP} \
            -f ${FRAC} -r -c |\
        gzip -c > ${FD_OUT}/${FN_OUT}
        
        ### Loop END Message
        echo "Show the first few lines of the output file"
        echo ${FD_OUT}/${FN_OUT}
        zcat ${FD_OUT}/${FN_OUT} | head -n 3
        echo
    done
done

### END message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27283779


In [18]:
scancel 27277389

In [14]:
cat ${FD_LOG}/fragment_input_score_per1.txt

Slurm Array Index: 
Time Stamp:         02-04-22+14:34:43

++++++++++++++++++++++++++++++++++++++++++++++++
Input file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1//target_PER1.bed

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1//target_PER1.bed
chr17	8148117	8149012
chr17	8148122	8149107
chr17	8148178	8149194

Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/input_score/Input1/target_PER1_f50_Input1.bed.gz
/var/spool/slurmd/job27277217/slurm_script: line 55: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/input_score/Input1/target_PER1_f50_Input1.bed.gz: No such file or directory
Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/input_score/Input1/target_PER1_f50_Input1.bed.gz
gzip: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/input_score/Input1/target_PER1_f50_Input1.bed.gz: No such file or directory

Output file:  /gpfs/fs1/data/reddylab/Kuei/out/p